In [1]:
import os
import pandas as pd
import dask.dataframe as dd
import numpy as np
import glob
from tqdm import tqdm

from data_import import *

##### test cells

In [17]:
os.listdir(data_dir)

NameError: name 'data_dir' is not defined

In [ ]:
[not(i.endswith('.csv')) for i in os.listdir(data_dir)]

[False, True, True, True]

##### data imports

In [ ]:
soil_data = pd.read_csv('./data/test_timeseries/test_timeseries.csv')

In [ ]:
oregon_test_data = soil_data[(soil_data['fips']>41000) & (soil_data['fips']<42000)].copy()

In [ ]:
soil_data_2 = dd.read_csv('./data/train_timeseries/train_timeseries.csv')

In [ ]:
testoregon = soil_data_2.loc[(soil_data_2['fips']>41000) & (soil_data_2['fips']<42000)]

In [ ]:
soil_data_3 = dd.read_csv('./data/test_timeseries/test_timeseries.csv')

In [ ]:
testoregon2 = soil_data_3.loc[(soil_data_3['fips']>41000) & (soil_data_3['fips']<42000)]

In [ ]:
testoregon.compute()

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,...,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
445869,41001,2000-01-01,1.69,85.30,2.73,-5.72,-6.46,-6.45,-2.15,-7.78,...,-6.84,2.09,2.87,1.24,1.63,3.62,5.60,2.03,3.57,NaN
445870,41001,2000-01-02,3.82,85.63,2.50,-6.29,-7.42,-7.40,-2.85,-8.82,...,-7.37,2.56,4.01,1.86,2.15,4.42,5.88,3.16,2.72,NaN
445871,41001,2000-01-03,5.89,86.65,2.54,-6.21,-7.20,-7.19,-2.64,-9.10,...,-7.18,1.99,2.82,0.44,2.38,3.02,4.23,0.75,3.48,NaN
445872,41001,2000-01-04,7.12,86.19,3.24,-3.61,-4.29,-4.28,0.23,-6.38,...,-4.14,3.26,4.57,2.02,2.55,5.43,7.52,2.98,4.54,0.0
445873,41001,2000-01-05,0.88,86.81,2.65,-5.81,-6.65,-6.64,-1.44,-10.00,...,-8.30,2.50,3.80,0.77,3.03,4.02,5.78,1.11,4.67,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100329,41071,2016-12-27,15.42,99.59,4.99,4.09,3.67,3.65,5.59,1.98,...,3.93,3.18,5.96,1.72,4.24,5.83,9.48,3.86,5.61,0.0
100330,41071,2016-12-28,0.74,100.37,4.06,1.88,0.84,0.85,5.33,-0.18,...,0.88,1.84,2.14,1.30,0.84,3.21,4.40,1.96,2.44,NaN
100331,41071,2016-12-29,1.62,99.82,4.30,2.21,1.44,1.44,5.46,-1.10,...,1.98,1.98,2.41,1.14,1.28,3.64,4.62,2.78,1.84,NaN
100332,41071,2016-12-30,0.78,99.97,4.29,2.18,1.57,1.57,5.23,-0.72,...,1.75,1.94,2.77,1.32,1.44,3.42,4.66,2.43,2.24,NaN


In [ ]:
len(testoregon2.compute())

26316

In [ ]:
soil_data.columns

Index(['fips', 'date', 'PRECTOT', 'PS', 'QV2M', 'T2M', 'T2MDEW', 'T2MWET',
       'T2M_MAX', 'T2M_MIN', 'T2M_RANGE', 'TS', 'WS10M', 'WS10M_MAX',
       'WS10M_MIN', 'WS10M_RANGE', 'WS50M', 'WS50M_MAX', 'WS50M_MIN',
       'WS50M_RANGE', 'score'],
      dtype='object')

In [ ]:
test = soil_data_2[(soil_data_2['fips']>41000) & (soil_data_2['fips']<42000)].copy()

AttributeError: Can only use .str accessor with string values!

In [ ]:
import pathlib

In [ ]:
def import_data_dask()-> dict:
    """
    This function imports raw data from directory using dask and returns a dictionary of dask DataFrames.

    Parameters
    ----------
    None.

    Returns
    -------
    'raw_data_dict' : dict
    """
    input_dir = os.path.abspath("data\\*\\*")

    data_paths = glob.glob(input_dir, recursive=True)

    raw_data_dict = {}

    for data in tqdm(data_paths, desc="file import"):
        name = os.path.basename(data).split('.')[0]
        raw_data_dict[name] = dd.read_csv(data)
    return raw_data_dict

In [ ]:
def oregon_only(data_dict: dict):
    """
    This function trims input data to only contain data regarding Oregon counties.

    Parameters
    ----------
    'data_dict' : dict
        This is a dictionary with train, test, and validation datasets.
    """
    oregon_data_dict = {}
    for keyval in tqdm(data_dict.keys(), desc='table management'):
        table = data_dict[keyval]
        table = table.loc[(table['fips']>41000) & (table['fips']<42000)].copy()
        table = table.compute()
        oregon_data_dict[keyval] = table
    
    return oregon_data_dict

In [ ]:
raw_data_dict = import_data_dask()

file import: 100%|██████████| 3/3 [00:00<00:00, 31.26it/s]


In [ ]:
oregon_data = oregon_only(raw_data_dict)

table management: 100%|██████████| 3/3 [00:22<00:00,  7.56s/it]


In [ ]:
train = raw_data_dict['train_timeseries'][(raw_data_dict['train_timeseries']['fips']>41000) & (raw_data_dict['train_timeseries']['fips']<42000)].copy()

In [ ]:
raw_data_dict['test_timeseries']

,fips,date,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
npartitions=4,,,,,,,,,,,,,,,,,,,,,
,int64,string,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
data_dir.keys()

##### Data Imports

In [2]:
data_importing = OregonImport()
data_dict = data_importing.oregon_data_runner()

table management: 100%|██████████| 3/3 [00:24<00:00,  8.04s/it]
